In [ ]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb

wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip -P /tmp/
unzip -o /tmp/chromedriver-linux64.zip -d /tmp/
chmod +x /tmp/chromedriver-linux64/chromedriver
mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
pip install selenium chromedriver_autoinstaller

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,974 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,398 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-secur

In [ ]:
!pip install htmlmin
!pip install openai
!pip install html2text
!pip install python-dotenv

  Preparing metadata (setup.py) ... done
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27080 sha256=ec0c23efdf824396e346b7b717f09aa214151f5ab00ee66bf6c4a1c8dd2fd4d8
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for html2text: filename=html2text-2024.2.26-py3-none-any.whl size=33111 sha256=3350b3384e7c08a9d90920142aa553855844e24cea5b0938bfea1e0a6c16f97f
  Stored in directory: /root/.cache/pip/wheels/f3/96/6d/a7eba8f80d31cbd188a2787b81514d82fc5ae6943c44777659
Successfully built html2text


In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller
from bs4 import BeautifulSoup
import json

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chromedriver_autoinstaller.install()

driver = webdriver.Chrome(options=chrome_options)

In [ ]:
def get_programs(driver):
    driver.get("https://www.canadavisa.com/canada-pnp-finder-and-tracker.html")
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table=soup.find_all("table","ui celled fixed sortable table")[0]
    headers = [th.get_text() for th in table.find('tr').find_all('th')]
    data = []

    for row in table.find_all('tr')[1:]:
        data.append([td.get_text() for td in row.find_all('td')])
    df = pd.DataFrame(data, columns=headers)

    for i, row in enumerate(table.find_all('tr')[1:]):
        row_data = []
        for idx, td in enumerate(row.find_all('td')):
            if idx == 1:
                link = td.find('a')
                if link:
                    df.loc[i, 'Stream Link'] = link['href']
    return df.to_dict(orient='records')


In [ ]:
programs=get_programs(driver)
pd.DataFrame(programs).to_csv('all_programs.csv')

In [ ]:
from urllib.parse import urlparse
def uri_validator(x):
    try:
        result = urlparse(x)
        return all([result.scheme, result.netloc])
    except AttributeError:
        return False

In [ ]:
titles=['Province',
        'Stream',
        'Substream',
        'Last Updated',
        'Education',
        'Age',
        'Work Experience',
        'Language Proficiency',
        'Other Criteria']

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv('API_KEY')
client = OpenAI(
    api_key = api_key,
)


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
from selenium.common.exceptions import WebDriverException
import time
import html2text

def get_program_info(index, driver, program):
    url = program['Stream Link']
    province = program['Province']

    if not uri_validator(url):
        return []

    retries = 5
    delay = 2
    soup = None

    for i in range(retries):
        try:
            driver.get(url)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            break
        except WebDriverException as e:
            if i == retries - 1:
                print(f"Failed to retrieve page after {retries} attempts: {e}")
                return []
            time.sleep(delay)
            delay *= 2

    if soup is None:
        return []


    article = soup.find('article')
    if not article:
        print("Article not found in HTML")
        return []

    # article = summarize(article, word_count=300)

    prompt = prompt = '''
    Extract the following visa eligibility information from the provided HTML content and format it into a validated JSON object and give it as a plain text, not a mark down.

    Province
    Stream
    Substream
    Education
    Age (in this format: {min acceptable age}-{max acceptable age} like 18-35)
    Work Experience (in JSON format: the key is "category name" and the value is the years or null if it isn't mentioned)
    Language Proficiency (in this format: {"category name" : "level"})
    Other Criteria
    Last Updated (in format yyyy-mm-dd)

    The HTML content may contain structured data with tags and classes defining each piece of information. Your task is to identify these patterns and extract the relevant data accurately.
    Make sure to handle cases where certain information might not be available or might be presented in a different format. Provide the extracted information in JSON format, with each entry containing the aforementioned fields.
    If any field is not available or cannot be extracted, indicate it as null. Ensure that the Age information is also in JSON format, and the value field of JSON indicates the age eligibility.
    Do not indicate the point of each field, it doesn't matter how much point they get by having some eligibility.
    '''

    # print(article.text)
    size = len(article.text.split())
    if size > 10000:
        try:
            article = html2text.html2text(article)
        except:
            pass
    prompt = prompt + "\n html content:\n" + article.text
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        )
    except:
        print("too long")
        return []

    return response.choices[0].message.content


In [ ]:
data = []
strings = []
for program in programs:
    json_string=get_program_info(0, driver, program)
    # print(json_string)
    if len(json_string):
        strings.append(json_string)
        try:
            py_object = json.loads(json_string)
            data.append(py_object)
            print("JSON loaded successfully!")
        except json.JSONDecodeError as e:
            print(f"JSONDecodeError: {e.msg}")
            print(f"Error at line {e.lineno}, column {e.colno}, char {e.pos}")
    else:
        strings.append(program)
data = pd.DataFrame(data)
data

[]
[]
[]
Alberta Express Entry Stream
Last updated: 
14 March 2023
The Alberta Express Entry Stream allows the province of Alberta to nominate qualified candidates from the Government of Canada's Express Entry pool. Under Alberta Express Entry Stream qualified candidates may be selected for nomination through general draws or if they highly skilled tech professionals under an Accelerated Tech pathway.Complete our FREE assessment to find out if you are eligible for Canadian immigration!Table of ContentsOverviewGeneral Express Entry Stream Selection CriteriaAccelerated Tech Pathway Selection CriteriaAccelerated Tech Pathway List of Eligible OccupationsContact us for AssistanceOverviewYou may get selected for the Alberta Advantage Immigrant Program, formerly known as the Alberta Immigrant Nominee Program (AINP), if you have strong ties to Alberta or if you can help support the government’s economic development and diversification priorities.If you are nominated by a Canadian province, you

,Province,Category / Stream,Program Status,Express Entry-Linked,Job Required,Draw History,Stream Link,Stream,Substream,Education,Age,Work Experience,Language Proficiency,Other Criteria,Last Updated
0,Alberta,Accelerated Tech Pathway,Accepting applications,Yes,Yes,See history,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alberta,Rural Entrepreneur,Coming soon,No,No,See history,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alberta,Rural Renewal,Coming soon,No,Yes,See history,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Alberta,NaN,NaN,NaN,NaN,NaN,NaN,Express Entry,Accelerated Tech Pathway,None,"{""min"": 18, ""max"": 35}","{'High-wage and low-wage occupations': None, '...",None,Minimum CRS score of 300,2023-03-14
4,Alberta,NaN,NaN,NaN,NaN,NaN,NaN,Opportunity Stream,None,Canadian high school diploma or equivalent,"{'min': 18, 'max': 35}",{'category name': '12 months in Alberta within...,{'Canadian Language Benchmark (CLB) score': '4'},{'Residency Status and Work Permit Requirement...,2023-03-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,Yukon,NaN,NaN,NaN,NaN,NaN,NaN,Yukon Express Entry,None,None,"{'min': None, 'max': None}","{'Skilled Worker': 'at least one year', 'Criti...",{'Business Nominee': 'Level 6 in IELTS or Leve...,None,2023-03-14
91,Yukon,NaN,NaN,NaN,NaN,NaN,NaN,Express Entry,None,None,"{'min': None, 'max': None}","{'Skilled Worker': 'at least one year', 'Criti...",{'Business Nominee': 'Level 6 in IELTS (Englis...,{'Business Nominee': {'minimum score on Nomine...,2023-03-14
92,Yukon,NaN,NaN,NaN,NaN,NaN,NaN,Skilled Worker Program,None,High school diploma,"{'min': None, 'max': None}",{'Related Work Experience': '1'},{'English': '6 in each of the four abilities'},None,2021-10-04
93,Yukon,NaN,NaN,NaN,NaN,NaN,NaN,Critical Impact Worker Program,None,High school diploma,"{'min': None, 'max': None}",{'related work experience': '6 months'},{'English': 'Minimum scores specified for CELP...,None,null


In [ ]:
data.to_csv('results.csv')

In [ ]:
import numpy as np
def is_safe(x):
    try:
        if x in [None, ''] or pd.isna(x):
            return False
    except:
        return True
    return True

def json_loads(x):

     try:
        result=json.loads(x)
     except  Exception as e:
        result=x
        print(e)
     return result

def safe_json_loads(x):
    if not is_safe(x):
        return []

    try:
        result=json.loads(x)
    except Exception as e:
       result=json_loads(str(x).replace("'","\"").replace("None","null").replace("\"s","'s"))
    return result

In [ ]:
def transform_data(data):
    transformed_rows = []

    for item in data:
        name = item.get('Name')
        substreams = item.get('Substream', [])

        for substream in substreams:
            transformed_row = {
                'Name': name,
                'Profile': substream.get('Profile'),
                'Education': substream.get('Education'),
                'Language Proficiency': substream.get('Language Proficiency'),
                'Other Criteria': substream.get('Other Criteria'),
                'Last Updated': substream.get('Last Updated')
            }
            transformed_rows.append(transformed_row)

    return pd.DataFrame(transformed_rows)

In [ ]:
from pandas import json_normalize

def parse_json_column(df, column):
    df[column] = df[column].apply(safe_json_loads)

    df[column] = df[column].apply(lambda x: list(x.items()) if isinstance(x, dict) else x)
    df_expanded = df.explode(column).reset_index(drop=True)

    # Normalize the JSON data into separate columns
    json_df = pd.json_normalize(df_expanded[column])
    # Combine the original dataframe with the new JSON columns
    # df_combined = df_expanded.drop(columns=[column]).join(json_df)
    df_combined = df_expanded.join(json_df,how="left")
    # df_combined = df_expanded.merge(json_df,how="left")

    return df_combined

In [ ]:
def worker(df, column):
    df_expanded_res=None
    for i in range(len(df)):
        try:
            res=parse_json_column(df[i:i+1], column)
            if df_expanded_res is None:
                df_expanded_res=res
            else:
                df_expanded_res=pd.concat([df_expanded_res,res],axis=0)

        except:
            pass
    return df_expanded_res

In [ ]:
df_expanded = worker(data, 'Substream')
df_expanded = worker(df_expanded, 'Other Criteria')
df_expanded = worker(df_expanded, 'Language Proficiency')
df_expanded = worker(df_expanded, 'Education')
df_expanded = worker(df_expanded, 'Work Experience')
df_expanded

In [ ]:
def is_points_tuple(x):
    return isinstance(x, tuple) and x[0] == 'Points'

mask = df_expanded.applymap(is_points_tuple).any(axis=1)
filtered_df = df_expanded[~mask]

In [ ]:
def replace_specific_value(cell):
    if cell == ('category name', 'level'):
        return None
    return cell

df = filtered_df.applymap(replace_specific_value)
df

In [ ]:
df.reset_index(inplace=True)
df = df.drop(columns='index', axis=1)
df.rename(columns={'Unnamed: 0' : 'Code'}, inplace=True)
df.to_csv('output.csv')